I passaggi da effettuare sono
- Corrispondenza punti 2D e 3D
- Trovare la matrice P
- Usare fattorizzazione RQ (Variante della fattorizzazione QR)
- Trovare K che è la matrice R della fattorizzazione precedente
- Mentre la matrice R di rotazione (da non confondere con la R di RQ) sarà la matrice Q di RQ
- Trovare t vettore colonna della traslazione

Import di tutte le librerie necessarie
Usiamo numpy per la gestione delle matrici generate dalle immagini e math per fare i calcoli 

In [1]:
import numpy as np
import math
import cv2

Iniziamo mappando i punti 3D nello spazio 2D attraverso dei vettori di punti.
Questi punti sono stati definiti dalla professoressa

In [19]:
def define_Points():
    n_corners = 14

    #3D points
    objp = np.zeros((n_corners,3), np.float32)

    #first face
    objp[1,:] =[7, 0, 0]
    objp[2,:] =[7, 0, 7]
    objp[3,:] =[0, 0, 7]
    objp[4,:] =[3, 0, 4]

    #second face
    objp[5,:] =[0, 7, 0]
    objp[6,:] =[0, 7, 7]
    objp[7,:] =[0, 4, 4]

    #top face
    objp[8,:] =[7, 7, 7]
    objp[9,:] =[4, 4, 7]

    #additional points
    objp[10,:] =[7, 0, 3]
    objp[11,:] =[0, 3, 7]
    objp[12,:] =[0, 7, 3]
    objp[13,:] =[7, 2, 7]

    #2D points
    imgp = np.zeros((n_corners,2), np.float32)

    imgp[0,:] = [103, 254]
    imgp[1,:] = [252, 212]
    imgp[2,:] = [258, 39]
    imgp[3,:] = [103, 61]
    imgp[4,:] = [173, 132]

    imgp[5,:] = [18, 196]
    imgp[6,:] = [14, 26]
    imgp[7,:] = [50, 120]

    imgp[8,:] = [155, 10]
    imgp[9,:] = [136, 29]

    #additional points
    imgp[10,:] = [254, 141]
    imgp[11,:] = [62, 46]
    imgp[12,:] = [16, 127]
    imgp[13,:] = [224, 27]
    
    return objp, imgp


Definiziamo poi una funzione che stampa sull'immagine i punti che abbiamo definito prima nella funzione define_points()

In [4]:
def show_points(imgp):
    # Let's show the selected points 
    img = cv2.imread("image_to_cal.png")
    img_to_show = img.copy()
    for p in imgp:
        img_to_show = cv2.circle(img_to_show, p.astype(np.int32), 3, (255, 0, 0) )

    cv2.imshow("img", img_to_show)
    cv2.waitKey(0)
    cv2.destroyAllWindows()    

Carichiamo quindi l'immagine e stampiamo tutto a schermo

In [20]:
img = cv2.imread("image_to_cal.png")
objp, imgp = define_Points()
show_points(imgp)

La variabile objp avrà quindi all'interno le coordinate nello spazio in una matrice 14,3 e che corrisponde alla matrice che contiene i punti tridimensionali e imgp è la matrice che contiene gli stessi punti ma nello spazio 2D dell'immagine

Iniziamo quindi i calcoli per trovare la matrice P

Per ognuna della coppia 2D-3D aggiungiamo la dimensione w e cerchiamo la soluzione

Il primo passo è aggiungere la dimensione w ai due insiemi di vettori

In [16]:
for i in range(0, len(imgp)):
    _2d = imgp[i]
    _2d = np.append(arr= _2d, values= 1)
    #print(_2d)

for i in range (0, len(objp)):
    _3d = objp[i]
    _3d = np.append(arr= _3d, values= 1)
    #print(_3d)

[103. 254.   1.]
[252. 212.   1.]
[258.  39.   1.]
[103.  61.   1.]
[173. 132.   1.]
[ 18. 196.   1.]
[14. 26.  1.]
[ 50. 120.   1.]
[155.  10.   1.]
[136.  29.   1.]
[254. 141.   1.]
[62. 46.  1.]
[ 16. 127.   1.]
[224.  27.   1.]
[0. 0. 0. 1.]
[7. 0. 0. 1.]
[7. 0. 7. 1.]
[0. 0. 7. 1.]
[3. 0. 4. 1.]
[0. 7. 0. 1.]
[0. 7. 7. 1.]
[0. 4. 4. 1.]
[7. 7. 7. 1.]
[4. 4. 7. 1.]
[7. 0. 3. 1.]
[0. 3. 7. 1.]
[0. 7. 3. 1.]
[7. 2. 7. 1.]


Definisco quindi la matrice con i valori p da trovare

non ho capito come fare quindi costruisco direttamente la matrice su cui applicare il metodo dei minimi quadrati

In [46]:
def calibrate(imgp, objp):
    num_points = len(objp)
    A = np.zeros((2 * num_points, 11))
    b = np.zeros((2 * num_points, 1))

    for i in range(num_points):
        X, Y, Z = objp[i]
        u, v = imgp[i]

        A[2 * i, :] = [X, Y, Z, 1, 0, 0, 0, 0, -u * X, -u * Y, -u * Z]
        A[2 * i + 1, :] = [0, 0, 0, 0, X, Y, Z, 1, -v * X, -v * Y, -v * Z]

        b[2 * i, 0] = u
        b[2 * i + 1, 0] = v
    
    x, residuo, rank, s = np.linalg.lstsq(A, b, rcond=None)

    print (x.shape)

    K = np.matrix( [ [x[0], x[1], x[2]],\
                   [x[4], x[5], x[6]], \
                   [[x[8], x[9], 1]] ] \
                )
    distorsione = np.array([x[3], x[7], 0, 0, 0])

    return K, distorsione  

print(calibrate(imgp, objp)) 


(11, 1)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.